# Preliminary use case for Moisture Budgets

## Monthly Mean Eddy Covariances from 6-hourly model output

### PURPOSE: Calculate the monthly mean eddy covariances, U'Q' and V'Q'  from 6-hourly data

    U,V are 6-hourly (lon,lat,lev) wind data on model sigma levels
     Q   is 6-hourly specific humidity data on model sigma levels
 
### BACKGROUND: 
    The vertically integrated monthly mean atmospheric moisture 
    budgets provide an accounting for the physical processes leading 
    to the net surface water flux, E-P (Evaporation minus Precipitation).
    The physical processes can be time filtered in time into mean and 
    eddy terms and spatially into convergence and advection terms.  We 
    can also separate time anomalies and trends into those resulting 
    from changes in atmospheric moisture and those resulting from changes 
    in the atmospheric circulation.  This allows diagnostic evaluation
    of the physical mechanisms of hydroclimate variability and change.
    See:
    
    __Seager, R. and N. Henderson, 2013: Diagnostic computation of moisture 
    budgets in the ERA-Interim Reanalysis with reference to analysis of 
    CMIP-archived atmospheric model data. J. Climate, 26: 7876 - 7901__
    
    The most computationally expensive part of this calculation is the 
    calculation of the covariance terms, U*Q and V*Q since these quantities 
    are generally not included in the standard variables in the CMIP 
    archives.  Ideally they would be calculated for every time step of 
    the simulation. In the absence of these actual covariances, we must 
    compute an approximation from the 6-hourly model output that is standard 
    for CMIP.  Since most of the moisture is in the lower part of the 
    atmosphere where the topography plays a significant role, the horizontal 
    differencing and vertical integrals must be treated carefully.
    
    This is a test case for the calculation of the covariances using only 
    the NCAR/CCMS4 historical CMIP5 simulation. Only one ensemble member 
    is available, r6i1p1, but 1.5Tbytes of data must be processed to 
    calculate the monthly mean covariances.

### DATA USED: 
    The CMIP5-CCSM-historical test data is located on cheyenne.ncar.edu:
    
    /glade/p/CMIP/CMIP5/output1/NCAR/CCSM4/historical/6hr/atmos/6hrLev/r6i1p1/latest/
    
    There are two forms of the ua,va and hus fields available, one set 
    on the model sigma vertical grid and the other interpolated to a reduced 
    set of pressure levels. We use the pressure level data since most modeling 
    groups do not provide data on the native vertical grid, but NCAR/CCMS4 
    provides only U and V, Q is not available. I have submitted
    a request, but they are still 'Looking into it'.
    
    This particular model has 6-hourly data from 1950-2005 on 26 levels for 
    only one ensemble member. They provide 4 files per model year and each 
    file for each variable is 2.1G.  Thus the data to be processed is
    about 1.5Tb - just for the historical runs.

### METHOD:  for all levels,latitudes and longitudes, 
    calculate <U'*Q'> = <U*Q> - <U>*<Q>

### NOTATION:  

     U  = 6-hourly U data 
    <U> = monthly mean U 
     U' = U - <U>



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
%matplotlib inline

In [ ]:
# There are four CCSM4 6-hourly data files per year, and the names must reflect the time boundaries
def ReadYear(syear,sdir,tdir,var):
    start = {'JFM':'010100', 'AMJ':'040100', 'JAS':'070100', 'OND':'100100'}
    stop  = {'JFM':'033118', 'AMJ':'063018', 'JAS':'093018', 'OND':'123118'}

    ncfile = []
    for season in ['JFM', 'AMJ', 'JAS', 'OND']:
        ncfile.append(sdir+var+'/'+var+tdir+syear+start[season]+'-'+syear+stop[season]+'.nc')
       
    return ncfile   

In [ ]:
# Uncomment next line for cheyenne:
#dir_base = '/glade/p/CMIP/CMIP5/output1/'

# This is my test directory on local machine:
dir_base = './'

center = "NCAR"; model = "CCSM4"
scenario = "historical"
freq = "6hr"; realm = "atmos"; dtype = "6hrLev"

nyears = 1
start_year = 1960

ddir = dir_base+center+'/'+model+'/'+scenario+'/'+freq+'/'+realm+'/'+dtype+'/'
for edir in os.listdir(ddir):
    for year in range(start_year,start_year+nyears):
#        print('starting year',year)
        syear = str(year)
        sdir = ddir+edir+'/latest/'
        tdir = '_6hrLev_'+model+'_'+scenario+'_'+edir+'_'
        hus_files = ReadYear(syear,sdir,tdir,'hus')
        ua_files  = ReadYear(syear,sdir,tdir,'ua')
        va_files  = ReadYear(syear,sdir,tdir,'va')
        
        first = 1
        for hf, uf in zip(hus_files, ua_files):
            ds = xr.auto_combine([xr.open_dataset(hf), xr.open_dataset(uf)])
            ds['qu'] = ds.hus*ds.ua
            
            qum = ds.qu.groupby('time.month').mean('time')
            qmum = ds.hus.groupby('time.month').mean('time') * ds.ua.groupby('time.month').mean('time')
            qdiff = qum-qmum
#            print(qdiff.month)
            if first:
#                print('first')
                qpup = qdiff
                first = 0
            else:
#                print('next')
                qpup = xr.concat([qpup,qdiff], dim='month')
                
        first = 1
        for hf, vf in zip(hus_files, va_files):
            ds = xr.auto_combine([xr.open_dataset(hf), xr.open_dataset(vf)])
            ds['qv'] = ds.hus*ds.va
            
            qvm = ds.qv.groupby('time.month').mean('time')
            qmvm = ds.hus.groupby('time.month').mean('time') * ds.va.groupby('time.month').mean('time')
            qdiff = qvm-qmvm
#            print(qdiff.month)
            if first:
#                print('first')
                qpvp = qdiff
                first = 0
            else:
#                print('next')
                qpvp = xr.concat([qpvp,qdiff], dim='month')
            

In [ ]:
arrays = [xr.DataArray(1, name='qpup'),xr.DataArray(1, name='qpvp')]
dsm = xr.merge(arrays)  
# but this Datset does not inherit the lon,lat,lev grids

# so will do explicitly:
# note that the new 'time' grid is in months and has lost knowledge of year - 
#    so will put in name of output file for now
dsm = xr.Dataset({'lon': ('lon', ds.lon), 'lat': ('lat', ds.lat), 'lev': ('lev', ds.lev), 
                  'month':('month',qpup.month),
                  'qpup': (['month','lev','lat','lon'], qpup), 
                  'qpvp': (['month','lev','lat','lon'], qpvp)})
dsm.to_netcdf('covar'+syear+'.nc')

In [ ]:
plt.figure(figsize=(8,3))
dp=dsm.qpvp.mean('month').sel(lev=1, method='nearest').plot()
plt.title(r'surface <$V^{\,\prime} Q^{\,\prime}>$')
plt.ylabel('latitude');plt.xlabel('longitude')
plt.show()